prompt:
我想用别人训练好的模型去分辨我这些图片里是什么动物，从而辅助我这个模型更理解我给不同动物图片打分的逻辑
有没有一个能识别动物的model我能用的

ResNet50 作为预训练的卷积基模型被用于提取图片的特征。通过将 include_top=False，我们去掉了原本用于分类的顶部全连接层，只保留了卷积层和池化层，提取出来的特征会传给后续的全连接层进行分类。

GlobalAveragePooling2D：这一层用于池化 ResNet50 输出的特征图，它会将每个通道的特征图缩减为一个单一的数值。这个操作帮助我们减少模型的参数量和计算量。

训练部分：数据被划分为训练集、验证集和测试集。然后用 fit() 训练模型，训练过程中可以通过 TensorBoard 可视化训练过程。

比较两个图片：通过调整图片大小并输入到模型中，比较模型对两张图片的预测结果，得出你更喜欢的图片。

1. 准备环境

In [25]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import preprocess_input

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

2. 加载图片和评分

In [2]:
# 加载CSV文件
df = pd.read_csv('dogs_manual_score.csv')

# 获取图片路径和对应的评分（更改为分类标签）
image_paths = df['Photo'].apply(lambda x: os.path.join('dog_pics', x)).tolist()
labels = df['Score'].apply(lambda x: 'like' if x == 1 else ('soso' if x == 0 else 'dont like')).tolist()

# 将分类标签转换为数字编码
label_map = {'like': 0, 'soso': 1, 'dont like': 2}
y_labels = np.array([label_map[label] for label in labels])


3. 加载和预处理图片

In [4]:
# 加载图片并进行预处理
def load_image(image_path):
    img = image.load_img(image_path, target_size=(256, 256))  # 读取并调整大小
    img_array = image.img_to_array(img)  # 转换为数组
    img_array = img_array / 255.0  # 归一化处理
    return img_array

# 处理所有图片
images = np.array([load_image(path) for path in image_paths])

# 拆分训练集和测试集（80% 训练，20% 测试）
X_train, X_test, y_train, y_test = train_test_split(images, y_labels, test_size=0.2, random_state=42)


4. 使用ResNet50提取图片特征

In [5]:
# 使用ResNet50提取图片特征
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# 冻结ResNet50的层（不进行训练）
for layer in base_model.layers:
    layer.trainable = False

In [6]:
# 定义特征提取模型
feature_extractor = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

# 提取训练和测试集的特征
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)


8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step


5. 训练模型

In [10]:
# 确保数据是float32
X_train_features = X_train_features.astype(np.float32)
X_test_features = X_test_features.astype(np.float32)

# 创建分类模型
score_model = tf.keras.Sequential([
    Input(shape=(X_train_features.shape[1],)),  # 输入层，指定特征的维度
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')  # 输出3个类别
])

# 编译模型
score_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 训练模型
history = score_model.fit(
    X_train_features, y_train,  # 使用提取的特征数据进行训练
    epochs=20,
    batch_size=32,
    validation_data=(X_test_features, y_test)
)

Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.6829 - loss: 0.9623 - val_accuracy: 1.0000 - val_loss: 5.3836e-08
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 1.0000 - loss: 1.7720e-08 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 1.0000 - los

6. 模型评估

In [16]:
# 评估模型
loss, accuracy = score_model.evaluate(X_test_features, y_test)
print(f"Model Loss: {loss}, Accuracy: {accuracy}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Model Loss: 0.0, Accuracy: 1.0


In [17]:
# 计算准确率
y_pred = np.argmax(score_model.predict(X_test_features), axis=1)  # 获取预测类别
accuracy = np.mean(y_pred == y_test)
print(f'Classification accuracy: {accuracy}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Classification accuracy: 1.0


7. 比较两张图片，预测哪个更符合你的喜好

In [34]:
# 加载和预处理图像
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(256, 256))  # 读取并调整大小
    img_array = image.img_to_array(img)  # 转换为数组
    img_array = img_array / 255.0  # 归一化处理
    img_array = np.expand_dims(img_array, axis=0)  # 增加批次维度
    img_array = preprocess_input(img_array)  # 使用ResNet50的预处理函数
    return img_array

# 用于预测单张图片的类别
def predict_image(img_path):
    # 加载和预处理图片
    img_array = load_and_preprocess_image(img_path)

    # 提取图片特征
    features = feature_extractor.predict(img_array)

    # 使用训练好的分类模型进行预测
    prediction = score_model.predict(features)
    
    # 获取预测的类别（最大概率的类）
    predicted_class = np.argmax(prediction, axis=1)
    
    # 反向映射类别数字为标签
    class_labels = {0: 'like', 1: 'soso', 2: 'dont like'}
    predicted_label = class_labels[predicted_class[0]]

    print(f"The image is predicted as: {predicted_label}")

# 测试新的图片
img_path = "E:/纽大/Useless Machines/Yiqi's Pics Bias/dog_likeOrNot/like/3361739.jpg"  # 替换为你要测试的图片路径
predict_image(img_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
The image is predicted as: dont like
